In [1]:
import random
random.seed(5)
import numpy as np
np.random.seed(5)
import os
os.environ['PYTHONHASHSEED']=str(5)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+

import scipy.io as scio
import cv2
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython.display import clear_output
from librosa.effects import preemphasis
from librosa.core import piptrack

#read videos and audio as training data for my own video encoder. If using vgg net check next cell
from os import listdir
from os.path import isfile, join, getmtime
from scipy.io import wavfile
from sklearn import preprocessing
from librosa import feature, effects
from scipy.signal import savgol_filter, resample
#to play audio
from IPython.display import Audio 

In [2]:
#compute mel genralized cepstral coeff
import pysptk as pysptk
afile='scratch/avc/lisa_data_opt/1107/1107_part1_keyed_4738.0_4792.0_mouth_audio.wav'
sr,gtaud = wavfile.read(afile)

num=np.int(np.shape(gtaud)[0]/1024)

mgc=[]
for i in range(num):
    x=gtaud[1024*i:1024*(i+1)+1]
    mgc_temp = pysptk.mgcep(x, itype=3)
    mgc.append(mgc_temp)

print(np.shape(mgc), np.shape(gtaud))

(21, 26) (21600,)


In [3]:
#compute MCD and use dtw to align
import pysptk as pysptk

#method 1
def mcd(C, C_hat):
    """C and C_hat are NumPy arrays of shape (T, D),
    representing mel-cepstral coefficients.

    """
    K = 10 / np.log(10) * np.sqrt(2)
    return K * np.mean(np.sqrt(np.sum((C - C_hat) ** 2, axis=1)))

#method 2: searches for minima using dynamic time warping if the two MFCCs are not aligned
logSpecDbConst = 10.0 / np.log(10.0) * np.sqrt(2.0)

#x,y should be float64
def logSpecDbDist(x, y):
    size = x.shape[0]
    assert y.shape[0] == size

    sumSqDiff = 0.0
    for k in range(size):
        diff = x[k] - y[k]
        sumSqDiff += diff * diff

    dist = np.sqrt(sumSqDiff) * logSpecDbConst
    return dist



def getCostMatrix(xs, ys, costFn):
    assert len(xs) > 0 and len(ys) > 0

    costMat = np.array([ [ costFn(x, y) for y in ys ] for x in xs ])
    assert np.shape(costMat) == (len(xs), len(ys))
    return costMat

def getCumCostMatrix(costMat):
    xSize, ySize = np.shape(costMat)

    cumMat = np.zeros((xSize + 1, ySize + 1))
    cumMat[0, 0] = 0.0
    cumMat[0, 1:] = float('inf')
    cumMat[1:, 0] = float('inf')
    for i in range(xSize):
        for j in range(ySize):
            cumMat[i + 1, j + 1] = min(
                cumMat[i, j],
                cumMat[i, j + 1],
                cumMat[i + 1, j]
            )
            cumMat[i + 1, j + 1] += costMat[i, j]

    return cumMat

def getBestPath(cumMat):
    xSize = np.shape(cumMat)[0] - 1
    ySize = np.shape(cumMat)[1] - 1
    assert xSize > 0 and ySize > 0

    i, j = xSize - 1, ySize - 1
    path = [(i, j)]
    while (i, j) != (0, 0):
        _, (i, j) = min(
            (cumMat[i, j], (i - 1, j - 1)),
            (cumMat[i, j + 1], (i - 1, j)),
            (cumMat[i + 1, j], (i, j - 1))
        )
        path.append((i, j))
    path.reverse()

    return path

def dtw(xs, ys, costFn):
    """Computes an alignment of minimum cost using dynamic time warping.
    A path is a sequence of (x-index, y-index) pairs corresponding to a pairing
    of frames in xs to frames in ys.
    The cost of a path is the sum of costFn applied to (x[i], y[j]) for each
    point (i, j) in the path.
    A path is valid if it is:
        - contiguous: neighbouring points on the path are never more than 1
          apart in either x-index or y-index
        - monotone: non-decreasing x-index as we move along the path, and
          similarly for y-index
        - complete: pairs the first frame of xs to the first frame of ys (i.e.
          it starts at (0, 0)) and pairs the last frame of xs to the last frame
          of ys
    Contiguous and monotone amount to saying that the following changes in
    (x-index, y-index) are allowed: (+0, +1), (+1, +0), (+1, +1).
    This function computes the minimum cost a valid path can have.
    Returns the minimum cost and a corresponding path.
    If there is more than one optimal path then one is chosen arbitrarily.
    """
    costMat = getCostMatrix(xs, ys, costFn)
    cumMat = getCumCostMatrix(costMat)
    minCost = cumMat[len(xs), len(ys)]
    path = getBestPath(cumMat)
    return minCost, path

if 0:
    #nat is #of frames x #of coeff
    # ignore 0th cepstral component

    afile='scratch/avc/lisa_data_opt/1107/1107_part1_keyed_4738.0_4792.0_mouth_audio.wav'
    sr,gtaud = wavfile.read(afile)

    string='_diff_1_'
    testid=900
    newlabels=np.load('labels_1100.mat.npy')


    degaudiofile='test_output/new_fixed_combined_audio_'+string+str(testid)+'_'+ newlabels[testid][0]+'.wav'
    sr1,degaud = wavfile.read(degaudiofile)



    num=np.int(np.shape(degaud)[0]/1024)-1
    print(np.shape(gtaud), np.shape(degaud), sr, sr1, num)
    costFn=logSpecDbDist
    frame=1
    nat=[]
    for i in range(num):
        x=gtaud[1024*i:1024*(i+1)]
        mgc_temp = pysptk.mgcep(x)
        nat.append(mgc_temp)

    synth=[]
    for i in range(num):
        x=degaud[1024*i:1024*(i+1)]
        mgc_temp = pysptk.mgcep(x)
        synth.append(mgc_temp)

    print(np.shape(nat), np.shape(synth))
    nat=np.array(nat)
    synth=np.array(synth)
    nat = nat[:, 1:]
    synth = synth[:, 1:]
    minCost, path = dtw(nat, synth, costFn)
    frames = len(nat)
    mcdscore=minCost/frames

    print(minCost/frames, frames, mcd(nat, synth))




In [26]:
minCost/frames

5.658957798065539

In [6]:
#compute pesq and stoi scores

from pesq import pesq
from pystoi import stoi
import parselmouth
from parselmouth.praat import call
from librosa import feature, effects
from sklearn import preprocessing

filelist=np.load('filelist_1100.mat.npy')
newlabels=np.load('labels_1100.mat.npy')
string='_diff_1_'
freq=25
shp=16384

## aligned - bestmodel in all 3 curriculum learning
string='_diff_1_spec_mse_fftlen_bestmodel_all3_aligned_interp_video_stretch_audio_695_framecount_80_melW_0_'

#without phase
#string='_diff_1_spec_mse_fftlen_bestmodel_all3_aligned_interp_video_stretch_audio_jawmouth_695_framecount_80_melW_0__mouth'

#curriculum: C1
string='_diff_1_spec_mse_fftlen_bestmodel_all3_C2_aligned_interp_video_stretch_audio_jawmouth_695_framecount_80_melW_0__mouth'


# clear style only
string='_diff_1_spec_mse_fftlen_bestmodel_all3_C3_clearOnly_aligned_interp_video_stretch_audio_jawmouth_832_framecount_80_melW_0__mouth'

# plain style only
string='_diff_1_spec_mse_fftlen_bestmodel_all3_C3_plainOnly_aligned_interp_video_stretch_audio_jawmouth_832_framecount_80_melW_0__mouth'




newstyles=np.load('styles_1100.mat.npy')
newstyles=newstyles-1 #change the range from 1-2 to 0-1
test_idx1 = np.where(newstyles[860:]==0)[0]
test_idx2 = np.where(newstyles[860:]==1)[0]




scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
scaler_std= preprocessing.StandardScaler()
scaler_max= preprocessing.MaxAbsScaler()
gl=1


#loop over two styles
for st in range(2):
    if (st==0):
        test_idx = test_idx1 #plain
    else:
        test_idx= test_idx2 #clear

    pesq_wb=[]
    pesq_nb=[]
    st=[]
    md=[]
    for j in range(np.shape(test_idx)[0]):#488, 860, 695
        testid=test_idx[j]#860
        #testid=860+i
        
        afile=filelist[testid]+'_audio.wav'
        #print(afile)
        sr,aud = wavfile.read(afile)
        
        
        aud, index=effects.trim(np.float32(aud), top_db=freq)
        aud=np.append(aud,np.zeros([1024]))
        sound = parselmouth.Sound(aud.T, sampling_frequency=sr)
        #create manipulation object
        manipulation = call(sound, "To Manipulation",0.01, 50, 200)

        ##extract durationtier and add a duration point
        duration_tier = call(manipulation, "Extract duration tier")
        duration = call(duration_tier, "Add point", sound.end_time, shp/(np.shape(aud)[0]*1.0))

        #replace the duration tier in the manipulation object
        call([duration_tier, manipulation], "Replace duration tier")

        #Publish resynthesis
        sound_octave_up = call(manipulation, "Get resynthesis (overlap-add)")

        aud=sound_octave_up.values.T
        len_aud=np.shape(aud)[0]  
        aud=np.reshape(aud[0:shp],(shp,1))
        aud=scaler_std.fit_transform(aud)
        gtaud=scaler_max.fit_transform(aud)[:,0]

        #print(np.shape(gtaud), np.shape(degaud))

        if gl:
            degaudiofile='test_output/new_gl_audio_'+string+str(testid)+'_'+ newlabels[testid][0]+'_112257.wav' 
        else:
            degaudiofile='test_output/new_fixed_combined_audio_'+string+str(testid)+'_'+ newlabels[testid][0]+'.wav'


        #degaudiofile='test_output/new_fixed_combined_audio_'+string+str(testid)+'_'+ newlabels[testid][0]+'.wav'
        #degaudiofile='test_output/new_gl_audio_'+string+str(testid)+'_'+ newlabels[testid][0]+'_112257.wav'


        sr,degaud = wavfile.read(degaudiofile)
        
        #changing sampling rate to 8000
        snum=np.int(np.shape(gtaud)[0]/sr*8000)
        gtaud=resample(gtaud,snum)
        snum=np.int(np.shape(degaud)[0]/sr*8000)
        degaud=resample(degaud,snum)
        sr=8000

        if(np.shape(degaud)[0] > np.shape(gtaud)[0]):
            degaud=degaud[0:np.shape(gtaud)[0]]
        elif(np.shape(degaud)[0] < np.shape(gtaud)[0]):
            gtaud=gtaud[0:np.shape(degaud)[0]]


        #compute pesq scores
        #pq_wb=pesq(sr, gtaud, degaud, 'wb')
        #pesq_wb.append(pq_wb)
        pq_nb=pesq(sr, gtaud, degaud, 'nb')
        pesq_nb.append(pq_nb)    

        # Clean and gen should have the same length, and be 1D
        dis = stoi.stoi(gtaud, degaud, sr, extended=False)
        st.append(dis)


        #Compute MCD
        num=np.int(np.shape(degaud)[0]/1024)-3
        #print(np.shape(gtaud), np.shape(degaud), sr, num)
        costFn=logSpecDbDist
        frame=1
        nat=[]
        for j in range(1,num):
            x=gtaud[1024*j:1024*(j+1)+1]
            mgc_temp = pysptk.mgcep(x)
            nat.append(mgc_temp)

        if 1:
            #print(np.shape(mgc_temp))
            #degaud=np.reshape(degaud[0:shp],(shp,1))
            synth=[]
            for kidx in range(1,num):
                try:
                    x=degaud[1024*kidx:1024*(kidx+1)+1]
                    #print(np.shape(x),x)
                    mgc_temp = pysptk.mgcep(x)
                except:
                    print("Error at: ", kidx)
                else:
                    synth.append(mgc_temp)

        #print(np.shape(nat), np.shape(synth))
        nat=np.array(nat)
        synth=np.array(synth)
        nat = nat[:, 1:]
        synth = synth[:, 1:]
        minCost, path = dtw(nat, synth, costFn)
        frames = len(nat)
        mcdscore=minCost/frames

        md.append(mcdscore)
        #print(minCost/frames, frames, mcd(nat, synth))



        #print(pq_wb,pq_nb,dis, mcdscore)
    print(np.mean(pesq_wb),np.mean(pesq_nb), np.mean(st), np.mean(md))

 

/cvmfs/soft.computecanada.ca/easybuild/software/2017/Core/scipy-stack/2019a/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/cvmfs/soft.computecanada.ca/easybuild/software/2017/Core/scipy-stack/2019a/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan 1.4186365986422682 0.5942879677410059 6.792189047425478
nan 1.455233533486076 0.6229904490476696 6.485146736585375


/cvmfs/soft.computecanada.ca/easybuild/software/2017/Core/scipy-stack/2019a/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/cvmfs/soft.computecanada.ca/easybuild/software/2017/Core/scipy-stack/2019a/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [11]:
##original sampling rate
#16000 (16256,) (16256,)
#1.154174844730775 1.4211980946735987 0.6171961416059495 4.8541880543280636
#1.1449278064396071 1.3946675233218981 0.5700294148645002 5.069398428199972



##8000 sampling rate
#1.0627180337905884 1.510602298390451 0.6167263074694858 6.093059361063034
#1.0627180337905884 1.4775270534598308 0.5693015208195913 6.4215783325850175


#Clear only + gl=1
1.0627180337905884 1.510602298390451 0.6167263074694858 6.093059361063034
1.0627180337905884 1.4775270534598308 0.5693015208195913 6.4215783325850175

#plain only. + gl=0 + 8khz
nan 1.4563097645417145 0.5973996217063048 6.507774103697079
nan 1.4806313582088637 0.6304549754070317 6.26497469567256

#plain only + gl=1 + 8khz
nan 1.4191815857721573 0.5938728427854977 6.782106576133127
nan 1.4569377868071847 0.6237056020214267 6.495566262427654

16000 (8128,) (8128,)
